In [10]:
!wget -q https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
!wget -q https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv

In [12]:
import pandas as pd

# load CSV
train_df = pd.read_csv("train.csv", header=None, names=["label","title","description"])
test_df  = pd.read_csv("test.csv",  header=None, names=["label","title","description"])

# merge title+description
train_df["text"] = train_df["title"].str.strip() + ". " + train_df["description"].str.strip()
test_df["text"]  = test_df["title"].str.strip()  + ". " + test_df["description"].str.strip()

In [13]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Tokenize a small batch
batch = list(train_df["text"].iloc[:8])
encoding = tokenizer(batch,
                     padding="max_length",
                     truncation=True,
                     max_length=128,
                     return_tensors="pt")

print("input_ids:", encoding["input_ids"].shape)

input_ids: torch.Size([8, 128])


In [14]:
import torch

torch.save(encoding, "tokenized_sample.pt")


In [17]:
from datasets import Dataset, DatasetDict

# Converting pandas to DataFrames
hf_train = Dataset.from_pandas(train_df[["text","label"]])
hf_test  = Dataset.from_pandas(test_df[["text","label"]])

# Tokenize with tokenize_batch
def tokenize_batch(batch):
    return tokenizer(
      batch["text"],
      padding="max_length",
      truncation=True,
      max_length=128,
    )


tokenized_train = hf_train.map(tokenize_batch, batched=True, remove_columns=["text"])
tokenized_test  = hf_train.map(tokenize_batch,  batched=True, remove_columns=["text"])

# seting to PyTorch tensors
tokenized_train.set_format("torch", columns=["input_ids","attention_mask","label"])
tokenized_test.set_format("torch",  columns=["input_ids","attention_mask","label"])

# Saving the full tokenized dataset
tokenized_train.save_to_disk("/content/tokenized_train")
tokenized_test.save_to_disk("/content/tokenized_test")

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/120000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/120000 [00:00<?, ? examples/s]